In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 95% !important; }</style>"))

In [76]:
import sys
import os
import re
import numpy as np
import pandas as pd

# from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
pd.set_option('display.max_colwidth', None)

In [77]:
from lib import util, embeds, fitter

In [108]:
# FIXME: add lines for git clone SemeEval2022 and Stitch data

In [5]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'

Load all the CSV files in dataframes.

In [25]:
frames = util.load_csv_dataframes(datapath)

In [30]:
frames.keys()

dict_keys(['dev_submission_format.csv', 'dev_gold.csv', 'eval_submission_format.csv', 'train_one_shot.csv', 'train_zero_shot.csv', 'dev.csv', 'eval.csv'])

In [32]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']

In [71]:
z_emb = embeds.get_embeddings(zdf)
z_emb_i = embeds.get_embeddings(zdf, append=['MWE'])

Average sentence length: 157.86
Appending to Target: MWE
Average sentence length: 170.16


In [70]:
multilingual_model = 'distiluse-base-multilingual-cased-v1'

Get sentence-transformers embeddings with the best method (appending MWE to the text, ignoring context).

The "best" method isn't actually completely true, as the original paper uses the "idiomatic principle" to encode the MWE, that is, using it as a single token when tokenizing.

In [73]:
z_emb_multi = embeds.get_embeddings(zdf, modelname=multilingual_model, append=['MWE'])

Appending to Target: MWE
Average sentence length: 170.16


In [74]:
d_emb_multi = embeds.get_embeddings(ddf, modelname=multilingual_model, append=['MWE'])

Appending to Target: MWE
Average sentence length: 164.48


Do a fitting with Logistic Regression.

In [102]:
z_score, z_probs, z_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], d_emb_multi, ddf_gold['Label'])

In [103]:
z_score

0.6143437077131259

In [104]:
dres = fitter.add_results(ddf, z_results, ddf_gold)

In [105]:
dres

,ID,Language,MWE,Previous,Target,Next,Label,Prediction,Correct
0,3652,EN,high life,"Does the plumbing predictably rebel, creating a 1,000-foot cascade inside the central utility shaft?",Are these interruptions of the good life a necessary condition of the high life?,"Not at all , says James von Klemperer, president of the architecture firm Kohn Pedersen Fox, which plants skyscrapers all over the world: “In a building that thin, this kind of thing can happen, but it shouldn’t.",1,1,True
1,11103,EN,high life,Let’s be honest – we would be chuffed if our bosses gave us a box of chocolates or a gift voucher as a Christmas treat.,"But for Australian fashion designer Abby Kheir, there's no reason not to treat her employees to a taste of the high life all-year round.","Speaking to the Daily Mail , the owner of Abyss by Abby explained how she doesn't mind splashing out on her staff members so they all feel valued.",1,0,False
2,84346,EN,high life,I already have the winning ticket.,"With that, I will be enjoying the pleasures of the high life knowing I earned money the hard way: Gambling with a bit of mega luck.","Charles Selle is a former News-Sun reporter, political editor and editor.",1,0,False
3,56279,EN,high life,"There were signs everywhere of cosy, comfortable indoor-wear leaving its mark on outdoor styles, with both Miuccia Prada and Raf Simons layering up looks over long-johns.","Fendi offered swaddling, belted coats resembling dressing gowns over warm trousers, all in materials promising soft relaxation as much as the adventurous high life.","Shawl collars and enveloping dressing-gown styles were also on show at Zegna, while even windbreaker brand K-Way blurred lines between indoor and outdoor wear.",1,0,False
4,17886,EN,high life,Yet one thing is clear.,"Rick Ross and Diddy exemplify the high life, two bosses who are, by all counts, richer than they've ever been. Check out the picture below, and be sure to show some love to some hip-hop moguls in their natural habitat.","And who knows -- maybe we'll see Puff on his ad-lib game when Rick Ross drops his new album, vocals you already know were laced with blade firmly in hand.",1,0,False
...,...,...,...,...,...,...,...,...,...
734,95336,PT,papel higiênico,"O presidente da Suzano Papel e Celulose, Walter Schalka, afirmou em entrevista à Bloomberg sobre o surgimento do problema de exportação de polpa de celulose, utilizada para fabricação de papel higiênico.","A Suzano Papel e Celulose é uma das 10 maiores produtoras de celulose do mercado global, realizando um terço de todas as entregas mundiais de matéria- prima para fabricação de papel higiênico.",Vale destacar que a Suzano já recebeu menos entregas em março do que esperava e teve de adiar algumas para abril.,1,1,True
735,20353,PT,papel higiênico,"Atualmente, poderiam surgir problemas com cargas em outros tipos de embalagens, como sacos, barris e caixas.","O presidente da Suzano Papel e Celulose, Walter Schalka, afirmou em entrevista à Bloomberg sobre o surgimento do problema de exportação de polpa de celulose, utilizada para fabricação de papel higiênico.","A Suzano Papel e Celulose é uma das 10 maiores produtoras de celulose do mercado global, realizando um terço de todas as entregas mundiais de matéria- prima para fabricação de papel higiênico.",1,0,False
736,54479,PT,papel higiênico,"Policiais penais do Complexo Penitenciário Dr Manoel Carvalho Neto (Copemcan), em São Cristóvão, prenderam em flagrante cinco mulheres que tentavam entregar drogas aos internos da unidade prisional.",O material entorpecente estava escondido em rolos de papel higiênico.,"O flagrante ocorreu na manhã desta sexta-feira, 16.",1,0,False
737,85941,PT,papel higiênico,O acidente ocorreu depois de fortes rajadas de vento,"No caso do papel higiênico, a escassez de contêineres que já afetava o setor coincide com o aperto na oferta global de celulose, que enfrenta no momento uma demanda forte puxada pelo maior consumo de papel higiênico e outros produtos lig

In [106]:
dres_counts = util.get_counts(dres)

Show the MWEs that the model gets wrong more than half of the time. Are there any patterns?

In [107]:
dres_counts[dres_counts['Pct correct'] < 0.5].sort_values(by=['Language','MWE'])

,Language,MWE,0 (Idiomatic),1 (Literal),Total,Pct literal,Pct correct
20,EN,bad hat,6,9,15,0.600000,0.400000
49,EN,bow tie,0,20,20,1.000000,0.300000
7,EN,cotton candy,2,11,13,0.846154,0.153846
10,EN,dry land,12,3,15,0.200000,0.466667
39,EN,economic aid,0,15,15,1.000000,0.200000
33,EN,pillow slip,0,14,14,1.000000,0.142857
29,EN,public service,9,9,18,0.500000,0.388889
2,EN,rice paper,15,1,16,0.062500,0.375000
6,EN,swan song,12,2,14,0.142857,0.357143
42,PT,alarme falso,0,15,15,1.000000,0.133333
